## Single run example of nanotribological properties workflow

### Background
- Using SAM as coating to reduce coefficient of friction and force of adhesion when shearing between two surface
- Many variables that can be varied and, hence, makes it a prime candidate for Computational Simulation
- Several MD studies focusing on different properties
    - Andrew Paper: backbone chain length, chain density, terminal groups
    - Current screening project: mixed composition 
    - Near future project: backbone chemitries
- The legacy encourage this research setup to be TRUE
    - Future research can continue this project by varing and/or adding more variables based on the original project
    
### Model
- Initialization of two surfaces coated with SAM - varying mixing composition, backbone chainlength, terminal groups
- Two surface was then energy minimized through a few steps (by LAMMPS and GROMACS)
- The two system is then compressed and sheared against each other (the top surface moving, and the bottom surface fixed)
- The output is analyzed with mdanalysis

### Simulation Details
- Initialze by MoSDeF
- Fix overlaps by LAMMPS
- Energy Minimize by GROMACS
- NVT Equilibrate by GROMACS
- Compress by GROMACS
- Shearing at 5nN, 15nN, 25nN by GROMACS
- Data Analysis with mdanalysis

#### System initialization

In [1]:
import mbuild
from util.functions import system_builder

/Users/daico007/Documents/Code/Simulation/mosdef36/foyer/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)
/Users/daico007/Documents/Code/Simulation/mosdef36/foyer/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _CH4
  'Creating custom element for {}'.format(element))
/Users/daico007/Documents/Code/Simulation/mosdef36/foyer/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _CH3
  'Creating custom element for {}'.format(element))
/Users/daico007/Documents/Code/Simulation/mosdef36/foyer/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _CH2
  'Creating custom element for {}'.format(element))
/Users/daico007/Documents/Code/Simulation/mosdef36/foyer/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom el

In [2]:
# Change into working directory
%cd example_simulation

/Users/daico007/Documents/Papers/TRUE/TRUE-nanotribology/workflow/example_simulation


In [3]:
structure = system_builder(seed=1)

/Users/daico007/anaconda3/envs/mosdef36/lib/python3.6/site-packages/pandas/io/parsers.py:2455: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/daico007/anaconda3/envs/mosdef36/lib/python3.6/site-packages/pandas/io/parsers.py:2458: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/daico007/Documents/Code/Simulation/mosdef36/ctools/ctools/lib/recipes/monolayer.py:82: UserWarning: 
 No fractions provided. Assuming a single chain type.
  warn("\n No fractions provided. Assuming a single chain type.")
/Users/daico007/Documents/Code/Simulation/mosdef36/ctools/ctools/lib/recipes/monolayer.py:85: UserWarning: 
 Adding 100 of chain <Alkylsilane 60 particles, non-periodic, 59 bonds, id: 4789512960>
  warn("\n Adding {} of chain {}".format(len(pattern), chains[-1]))
/Users/daico007/Documents/Code/Simulation/mosdef36/ctools/ctools/lib/recipes/monolayer.py:85: UserWarning: 
 Adding 100 of chain <Alkyls

No urey bradley terms detected, will use angle_style harmonic
RB Torsions detected, will use dihedral_style opls
bottom_frozen: 623
top_frozen: 623
bottom_surface: 2132
top_surface: 2132
surfaces: 4264
bottom_chains: 6000
top_chains: 6000
chains: 12000
bottom_termini: 400
top_termini: 400
terminal_groups: 800
bottom: 8132
top: 8132
System: 16264


In [4]:
structure.visualize()

/Users/daico007/Documents/Code/Simulation/mosdef36/mbuild/mbuild/compound.py:2416: UserWarning: Guessing that "<OS pos=( 4.6333, 2.1916, 1.4235), 0 bonds, id: 4981511896>" is element: "O"
  atom, element))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#### Fix overlaps by LAMMPS

In [7]:
# Command (lmp_mpi) might be different (lmp) depend on the installed lammps version
%lmp_mpi -in ../util/mdp_files/in.minimize -log minimize.log

UsageError: Line magic function `%lmp_mpi` not found.


#### Energy minimize by GROMACS

In [6]:
# Convert .xtc to .gro
%gmx trjconv -s init.gro -f minimize.xtc -o minimize.gro  -b 1.0 -e 1.0
# Grompp
%gmx grompp -f util/mdp_files/em.mdp -c minimize.gro -p init.top -n init.ndx -o em.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -v -deffnm em -s em.tpr -cpi em.cpt -ntomp 1 -gpu_id 0000000000000000

UsageError: Line magic function `%gmx` not found.


#### NVT Equilibrate by GROMACS

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/nvt.mdp -c em.gro -p init.top -n init.ndx -o nvt.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -v -deffnm nvt -s em.tpr -cpi em.cpt -ntomp 1 -gpu_id 0000000000000000

#### Compress wih GROMACS

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/compress.mdp -c nvt.gro -p init.top -n init.ndx -o compress.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -v -deffnm compress -s em.tpr -cpi em.cpt -ntomp 1 -gpu_id 0000000000000000

#### Shearing at 5nN

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/shear_5nN.mdp -c compress.gro -p init.top -n init.ndx -o shear_5nN.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -s shear_5nN.tpr -deffnm shear_5nN -cpi shear_5nN.cpt -cpo shear_5nN.cpt -noappend -ntomp 1 -gpu_id 000000000000000

#### Shearing at 15nN

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/shear_5nN.mdp -c compress.gro -p init.top -n init.ndx -o shear_5nN.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -s shear_15nN.tpr -deffnm shear_15nN -cpi shear_15nN.cpt -cpo shear_15nN.cpt -noappend -ntomp 1 -gpu_id 000000000000000

#### Shearing at 25nN

In [ ]:
# Grompp
%gmx grompp -f util/mdp_files/shear_5nN.mdp -c compress.gro -p init.top -n init.ndx -o shear_5nN.tpr -maxwarn 1
# Run GROMACS
%gmx mdrun -s shear_25nN.tpr -deffnm shear_25nN -cpi shear_25nN.cpt -cpo shear_25nN.cpt -noappend -ntomp 1 -gpu_id 000000000000000

#### Calculate nematic order

In [ ]:
# Unwrap stuff

# Run COF and F0 calculation